# **Piano Generation using Deep Learning**

## **1. Introduction**

In today's age, generative models are becoming more and more popular. The constant improvement in science and computing power have made it possible to do things we couldn't even imagine less than a decade ago. In this notebook, we are going to explore the various ways we can generate piano music using Deep Learning methods.

## **2. Data used**

### **2.1 MAESTRO dataset**

In this project, we will use the MAESTRO dataset. It consists of a total of 1276 songs with an overall duration of 198.7 hours in both wav and MIDI format. Due to resource constraints, we are only going to use the latter. The data is collected from numerous series of a piano e-competition, spanning from 2004 to 2018. The songs have been split in train, test and validation sets for us in a way, that no song is in more than one of the splits. The splits have been done in the proportion 80/10/10.

### **2.2 Data representation**

Throughout the experiments, the data was tokenized in numerous ways. First, for the training of the GRU and LSTM architectures, the songs were transferred into piano roll representation and split into sequences with length of 100 with padding at the end if the length is shorter. Due to memory constraints, when `__getitem__` is called, the dataset class returns a random sequece of length 100, instead of all possible sequences with length of 100 for the particular song. When it came to fine-tuning the DistilGPT-2 model, a different approach was used. All of the MIDI sequences were tokenized into words, which DistilGPT-2 as a language model can understand. Augmented data was also used to train the DistilGPT-2 model. For the experiments with the GAN architecture, third method of tokenizationn was used. It's inspired by the paper [C-RNN-GAN: Continuous recurrent neural networks with adversarial training (Olof Morgen)](https://arxiv.org/abs/1611.09904). Each tone is modeled with a quadruplet of *tone length, frequency, intensity* and *time spent since the previous tone*.

Throughout the experiments, different methods of tokenizing data were used. Here is a quick overview of them:

#### **2.2.1 Piano-roll representation**

The MIDI data is loaded into a Dataset, where the songs are converted into `muspy.Music` objects. Upon indexing, the datasets converts the song into piano-roll representation, which is a tensor of shape `(time_steps, 128)`, where `time_steps` is the number of time steps for the different tones and 128 is the number of possible pitch values.

#### **2.2.2 Word tokenization**

For the word tokenization, the library `miditok` was used. The tokenizer used for this experiment is called "REMI", introduced in the paper [Pop Music Transformer (Huang and Yang)](https://dl.acm.org/doi/10.1145/3394171.3413671). It converts tones into strings, that the DistilGPT-2 model can understand and can generate.

#### **2.2.3 Quadruplet representation**
The library `mido` was used to split the MIDI files into 4 continuous values per note: *tone length, frequency, intensity* and *time spent since the previous tone*. This method was used for the GAN architecture, which did not go down successfully.

## **3. Models**

Throughout the experiments, many different models and architectures were used. In this section, we dive deeper into each of them:

### **3.1 GRU and LSTM**

For the GRU and LSTM architectures, the pianoroll representation of the data was used. Experiments were performed with three different architecture types: Many-to-One, Many-to-Many and Encoder-Decoder. Here is a deep dive into all of them:

#### **3.1.1 Many-to-one**

The experiments done with the GRU and the LSTM models are almost identical. Each of the variants for the Many-to-one architecture expects an input size of 128, which is the number of tones in the piano representation format for each time step. Then, the input is passed onto 2 recurrent layers of GRU or LSTM with a hidden size of 256. In the end, the model returns one 128-dimensional vector, representing the next timestep in the piano roll. The dataset class is implemented in a way, that `__getitem__` returns a tuple of `(sequence, target)`, where `sequence` is a tensor of shape `(seq_len, 128)`, which is a piano roll of `seq_len` timesteps and `target` is a 128-dimensional vector, representing the next tone. The experiments are performed and explained in more detail in the notebooks `many_to_one_gru_training.ipynb` and `many_to_one_lstm_training.ipynb` in the "notebooks" folder.

#### **3.1.2 Many-to-many**

Again, the GRU and LSTM models, used for the Many-to-many variant are almost the same. The architecture is essentially the same, the main difference is its return value and the dataset implementation. Now the network returns not only its last output, but every output. The dataset is changed in a way, that it now returns a tuple of `(sequence, target)`, but this time `target` is a tensor of shape `(seq_len, 128)` and it represents the latter half of `sequence` and the next `seq_len / 2` tones, that the model has to generate. Again, further details can be seen in the `many_to_many_gru_training.ipynb` and `many_to_many_lstm_training.ipynb` notebooks. In the end, the loss didn't go down after several epochs so these models have been ditched.

#### **3.1.3 Encoder-decoder**

Due to the close similarity in the performance of the LSTM and the GRU cells in the previous experiments, the encoder-decoder architecture was done only with LSTM cells. The model is split into two parts - an encoder and a decoder. The encoder has an input size of 128 and 2 layers of bidirectional LSTM cells with a hidden size of 1024. The model returns the sum of the values for the hidden cells in both directions.

The decoder consists of an LSTM layer, which takes the encoder's outputs and projects them in 128 dimensions with a linear layer.

The two models are combined in a single model, which takes the given sequence, passes it to the encoder and generates a `seq_len`-long output with a decoding loop. The model also uses teacher forcing, which forces the model to stay closer to the "ground-truth", while learning.

Further details can be seen in the `encoder_decoder_lstm_training.ipynb` notebook.

### **3.2 Pre-trained GPT-2**
For this experiment, "DistilGPT-2" was used. This is a pre-trained distilled version of OpenAI's "GPT2" model from HuggingFace. A pre-trained model was loaded and was fine-tuned for 10 epochs with the musical tokens from the REMI tokenizer. Overfitting was observed very early on.

The whole process can be observed in the `pretrained_gpt2_finetuning.ipynb` notebook.

### **3.3 GAN architecture**
Experiments were also conducted with a GAN architecture. 

The generator is a simple neural network, which takes 100 random numbers, passes them through 2 LSTM layers with a hidden size of 350, as inspired from [Olof Morgen's work](https://arxiv.org/abs/1611.09904). Then, the results are passed onto a Linear layer to project them onto a size of 4.

The discriminator is again a simple network, which takes a track, passes it through 2 Bi-directional LSTM layers with a hidden size of 350 and then a Linear layer to project them into 1 dimension and make the decision.

The training is in a cat-and-mouse style, where the generator tries to fool the discriminator by producing samples, that it can't distinguish and the discriminator tries to become better at spotting fake tracks.

During the training process, an error popped up, that I couldn't resolve, so in the end, it was not trained.
All of the details are in the `gan_experiments.ipynb` notebook.

## **4. Results**
### **4.1 Objective Measures**
To test the performance of the models, a few objective measures were taken, namely **pitch diversity**, **rhythmic consistency** and **note density**. None of the models have come really close to the test data in terms of these measures, but the differences between the models becomes apparent. The overall winner in these metrics is the DistilGPT-2 model, but the Many-to-One GRU has come really close to it on the rhythmic entropy measure and the Many-to-One LSTM even beat it in the note density measure.
### **4.2 Subjective Measures**
After listening to samples from all of the models, the clear winner is the DistilGPT-2, which is not surprising at all, given how large it is, compared to every other model. It produces some samples, that are really not that bad, although some of them are far from convincing. The other models sound really bad with random pauses and random tone bursts. This proves the point, that in order to model music, you need a way larger model.

More detailed measures can be seen in the `music_evaluation.ipynb` notebook.

## **5. Conclusion**
There is still much room for improvement for these models and this is one of the most simple ways one can model music. Future work could focus on implementing "attention" to the Encoder-Decoder architecture and more experiments with GAN architectures.

## **6. References**
1. Curtis Hawthorne, Andriy Stasyuk, Adam Roberts, Ian Simon, Cheng-Zhi Anna Huang,
  Sander Dieleman, Erich Elsen, Jesse Engel, and Douglas Eck. *"Enabling
  Factorized Piano Music Modeling and Generation with the MAESTRO Dataset."*
  In International Conference on Learning Representations, 2019.

2. Olof Morgen. *"C-RNN-GAN: Continuous recurrent neural networks with adversarial training"*

3. Yu-Siang Huang, Yi-Hsuan Yang. *"Pop Music Transformer: Beat-based Modeling and Generation of Expressive Pop Piano Compositions"*

4. Sanh, Victor and Debut, Lysandre and Chaumond, Julien and Wolf, Thomas. *"DistilBERT, a distilled version of BERT: smaller, faster, cheaper and lighter"*